# 02. Data Preprocessing Pipeline

**Mục tiêu**: Thực hiện pipeline tiền xử lý cho 3 thuật toán: Linear Regression, SVR, XGBoost

**Tương ứng Report Section 3**: Phương pháp & Pipeline Tiền xử lý

---

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

import sys
import os
sys.path.append(os.path.abspath('../src'))

print("Thư viện đã được tải thành công!")

Thư viện đã được tải thành công!


## 2.1 Tải dữ liệu gốc

In [2]:
# Tải dữ liệu gốc
df = pd.read_csv('../data/raw/global-data-on-sustainable-energy.csv')  
print(f"Kích thước dữ liệu gốc: {df.shape}")

# Chuyển đổi các số dạng chuỗi có dấu phẩy sang float
for col in df.columns:
    if df[col].dtype == 'object' and col not in ['Entity']:
        try:
            df[col] = df[col].str.replace(',', '').astype(float)
        except:
            pass

print(f"Các cột: {df.columns.tolist()}")
df.head()

Kích thước dữ liệu gốc: (3649, 21)
Các cột: ['Entity', 'Year', 'Access to electricity (% of population)', 'Access to clean fuels for cooking', 'Renewable-electricity-generating-capacity-per-capita', 'Financial flows to developing countries (US $)', 'Renewable energy share in the total final energy consumption (%)', 'Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)', 'Electricity from renewables (TWh)', 'Low-carbon electricity (% electricity)', 'Primary energy consumption per capita (kWh/person)', 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)', 'Value_co2_emissions_kt_by_country', 'Renewables (% equivalent primary energy)', 'gdp_growth', 'gdp_per_capita', 'Density\\n(P/Km2)', 'Land Area(Km2)', 'Latitude', 'Longitude']


,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60.0,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60.0,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60.0,652230.0,33.93911,67.709953


## 2.2 Tiền xử lý chung

Các bước chung cho tất cả thuật toán:
1. Xử lý Missing Values (Median Imputation)
2. Tạo Lag Features
3. Loại bỏ năm 2020 (COVID anomaly)

In [3]:
TARGET = 'Value_co2_emissions_kt_by_country'

# Whitelist các nền kinh tế lớn (G20+ và các nước phát thải lớn)
WHITELIST = [
    'China', 'United States', 'India', 'Russia', 'Japan', 'Germany', 
    'South Korea', 'Iran', 'Saudi Arabia', 'Indonesia', 'Canada', 
    'Mexico', 'South Africa', 'Brazil', 'Australia', 'Turkey', 
    'United Kingdom', 'France', 'Ítaly', 'Poland', 'Taiwan', 
    'Thailand', 'Spain', 'Malaysia', 'Egypt', 'Vietnăm', 'Pakistan',
    'Argentina', 'Venezuela', 'United Arab Emirates', 'Netherlands',
    'Iraq', 'Philippines', 'Kazakhstan', 'Algeria', 'Kuwait', 
    'Belgium', 'Czechia', 'Morocco'
]

# 1. Median Imputation
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
imputer = SimpleImputer(strategy='median')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
print(f"Missing values sau imputation: {df.isnull().sum().sum()}")

# 2. Tạo Lag Features
lag_cols = [TARGET, 'gdp_per_capita', 'Primary energy consumption per capita (kWh/person)']
for col in lag_cols:
    if col in df.columns:
        df[f'{col}_lag1'] = df.groupby('Entity')[col].shift(1)

# Loại bỏ năm đầu tiên của mỗi quốc gia (không có lag)
df = df.dropna(subset=[f'{TARGET}_lag1'])
print(f"Kích thước sau khi tạo lag: {df.shape}")

# Lưu dữ liệu đã tiền xử lý chung
df.to_csv('../data/processed/common_preprocessed.csv', index=False)
print(f"Đã lưu common_preprocessed.csv: {df.shape}")

Missing values sau imputation: 0
Kích thước sau khi tạo lag: (3473, 24)
Đã lưu common_preprocessed.csv: (3473, 24)


## 2.3 Tiền xử lý cho Linear Regression

Đặc biệt cho LR:
- **Log Transform** cho skewed features (giảm ảnh hưởng của extreme values)
- **One-Hot Encoding** cho Entity (LR cần dummy variables)
- **IQR Outlier Removal** với Whitelist các nền kinh tế lớn
- **StandardScaler** (Z-Score Scaling)

In [4]:
df_lr = df.copy()

# Loại bỏ năm 2020 (COVID anomaly)
df_lr = df_lr[df_lr['Year'] != 2020]
print(f"Sau khi loại bỏ 2020: {df_lr.shape}")

# Log Transform cho các features bị lệch
skewed_cols = ['Financial flows to developing countries (US $)', 
               'Electricity from fossil fuels (TWh)',
               'Electricity from nuclear (TWh)',
               'Electricity from renewables (TWh)',
               ]

for col in skewed_cols:
    if col in df_lr.columns:
        # log1p để xử lý giá trị 0
        df_lr[col] = np.log1p(df_lr[col].clip(lower=0))

print(f"Đã áp dụng log transform cho {len(skewed_cols)} cột")

# Loại bỏ Outlier bằng IQR (ngoại trừ whitelist)
Q1 = df_lr[TARGET].quantile(0.25)
Q3 = df_lr[TARGET].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outlier_mask = ((df_lr[TARGET] < lower) | (df_lr[TARGET] > upper))
whitelist_mask = df_lr['Entity'].isin(WHITELIST)
df_lr = df_lr[~outlier_mask | whitelist_mask]
print(f"Sau khi loại bỏ outlier: {df_lr.shape}")

# One-Hot Encoding
df_lr = pd.get_dummies(df_lr, columns=['Entity'], prefix='Entity')

# StandardScaler (Z-Score) - loại trừ các cột binary
scale_cols = [c for c in df_lr.columns if c not in [TARGET, 'Year'] and not c.startswith('Entity_')]
scaler_lr = StandardScaler()
df_lr[scale_cols] = scaler_lr.fit_transform(df_lr[scale_cols])

print(f"Kích thước cuối cùng LR: {df_lr.shape}")

Sau khi loại bỏ 2020: (3298, 24)
Đã áp dụng log transform cho 4 cột
Sau khi loại bỏ outlier: (3260, 24)
Kích thước cuối cùng LR: (3260, 196)


## 2.4 Tiền xử lý cho SVR

Đặc biệt cho SVR:
- **Log Transform** cho skewed features (giống LR)
- **Ordinal Encoding** cho Entity (tiết kiệm memory, SVR không cần One-Hot)
- **IQR Outlier Removal** với Whitelist (SVR rất nhạy cảm với outliers)
- **RobustScaler** (tốt hơn StandardScaler vì dùng median/IQR thay vì mean/std)

In [5]:
df_svr = df.copy()

# Loại bỏ năm 2020 (COVID anomaly)
df_svr = df_svr[df_svr['Year'] != 2020]
print(f"Sau khi loại bỏ 2020: {df_svr.shape}")

# Log Transform cho các features bị lệch (giống LR)
for col in skewed_cols:
    if col in df_svr.columns:
        df_svr[col] = np.log1p(df_svr[col].clip(lower=0))

print(f"Đã áp dụng log transform cho {len(skewed_cols)} cột")

# Loại bỏ Outlier bằng IQR (ngoại trừ whitelist)
Q1 = df_svr[TARGET].quantile(0.25)
Q3 = df_svr[TARGET].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outlier_mask = ((df_svr[TARGET] < lower) | (df_svr[TARGET] > upper))
whitelist_mask = df_svr['Entity'].isin(WHITELIST)
df_svr = df_svr[~outlier_mask | whitelist_mask]
print(f"Sau khi loại bỏ outlier: {df_svr.shape}")

# Ordinal Encoding (giống XGBoost)
entity_map_svr = {e: i for i, e in enumerate(df_svr['Entity'].unique())}
df_svr['Entity_Encoded'] = df_svr['Entity'].map(entity_map_svr)
df_svr = df_svr.drop('Entity', axis=1)

# RobustScaler (tốt hơn StandardScaler cho SVR)
# RobustScaler dùng median và IQR, ít bị ảnh hưởng bởi outliers còn sót
scale_cols_svr = [c for c in df_svr.columns if c not in [TARGET, 'Year', 'Entity_Encoded']]
scaler_svr = RobustScaler()
df_svr[scale_cols_svr] = scaler_svr.fit_transform(df_svr[scale_cols_svr])

print(f"Kích thước cuối cùng SVR: {df_svr.shape}")

Sau khi loại bỏ 2020: (3298, 24)
Đã áp dụng log transform cho 4 cột
Sau khi loại bỏ outlier: (3260, 24)
Kích thước cuối cùng SVR: (3260, 24)


## 2.5 Tiền xử lý cho XGBoost

Đặc biệt cho XGBoost:
- **Không cần Log Transform** (tree-based tự xử lý skewness)
- **Ordinal Encoding** cho Entity
- **Không loại bỏ outliers** (tree-based robust với outliers)
- **Không cần scaling** (tree-based không phụ thuộc vào scale)

In [6]:
df_xgb = df.copy()

# Ordinal Encoding
entity_map_xgb = {e: i for i, e in enumerate(df_xgb['Entity'].unique())}
df_xgb['Entity_Encoded'] = df_xgb['Entity'].map(entity_map_xgb)
df_xgb = df_xgb.drop('Entity', axis=1)

# Không cần scaling, không cần loại outliers
print(f"Kích thước cuối cùng XGBoost: {df_xgb.shape}")

Kích thước cuối cùng XGBoost: (3473, 24)


## 2.6 Lưu dữ liệu đã xử lý

In [7]:
# Lưu tất cả 3 datasets
df_lr.to_csv('../data/processed/lr_final_prep.csv', index=False)
df_svr.to_csv('../data/processed/svr_final_prep.csv', index=False)
df_xgb.to_csv('../data/processed/xgb_final_prep.csv', index=False)

print("✅ Đã lưu dữ liệu đã tiền xử lý!")
print(f"  - lr_final_prep.csv: {df_lr.shape}")
print(f"  - svr_final_prep.csv: {df_svr.shape}")
print(f"  - xgb_final_prep.csv: {df_xgb.shape}")

✅ Đã lưu dữ liệu đã tiền xử lý!
  - lr_final_prep.csv: (3260, 196)
  - svr_final_prep.csv: (3260, 24)
  - xgb_final_prep.csv: (3473, 24)


## Tổng kết

**So sánh Pipeline tiền xử lý:**

| Thuật toán | Log Transform | Encoding | Xử lý Outlier | Scaling |
|---|---|---|---|---|
| **Linear Regression** | ✅ Có (skewed features) | One-Hot | IQR + Whitelist | StandardScaler |
| **SVR** | ✅ Có (skewed features) | Ordinal | IQR + Whitelist | RobustScaler |
| **XGBoost** | ❌ Không | Ordinal | Không (giữ tất cả) | Không |

**Các bước chung (tất cả thuật toán):**
- Median Imputation cho missing values
- Lag Features: CO2_lag1, GDP_lag1, Energy_lag1

**Điểm khác biệt chính:**
- **LR**: One-Hot Encoding → nhiều cột, StandardScaler (mean=0, std=1)
- **SVR**: RobustScaler (dùng median/IQR) → ít nhạy cảm với outliers còn sót
- **XGBoost**: Tree-based → không cần transform, tự xử lý non-linear patterns
- **Outliers**: LR & SVR loại bỏ outliers nhưng giữ các nền kinh tế lớn (G20+)

**File đầu ra:**
- `common_preprocessed.csv`: Dữ liệu chung sau imputation + lag features
- `lr_final_prep.csv`: Dữ liệu cho Linear Regression
- `svr_final_prep.csv`: Dữ liệu cho SVR
- `xgb_final_prep.csv`: Dữ liệu cho XGBoost